In [2]:
import os
import cv2
import time
import matplotlib.pyplot as plt

def detect_and_match_features(image_paths):
    # Initialize OpenCV feature detector (SIFT, ORB, etc.) and matcher (Brute Force, FLANN, etc.)
    detector = cv2.SIFT_create()
    matcher = cv2.BFMatcher()

    # Initialize lists to store keypoints and descriptors for each image
    keypoints_list = []
    descriptors_list = []

    # Detect keypoints and compute descriptors for each image
    for image_path in image_paths:
        # Read the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Detect keypoints and compute descriptors
        keypoints, descriptors = detector.detectAndCompute(image, None)

        # Add keypoints and descriptors to the lists
        keypoints_list.append(keypoints)
        descriptors_list.append(descriptors)

    # Match features between images
    matches = []
    for i in range(len(descriptors_list) - 1):
        # Match descriptors between consecutive pairs of images
        matches.append(matcher.match(descriptors_list[i], descriptors_list[i + 1]))

    return keypoints_list, matches

def find_image_paths(root_dir, scene_category):
    image_paths = []
    scene_dir = os.path.join(root_dir, scene_category.replace(" ", "_"))
    if os.path.isdir(scene_dir):
        for subdir, _, files in os.walk(scene_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(subdir, file))
    return image_paths

def visualize_matches(image_paths, keypoints_list, matches):
    for i, match in enumerate(matches):
        # Read images for the current pair of matches
        image1 = cv2.imread(image_paths[i], cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(image_paths[i + 1], cv2.IMREAD_GRAYSCALE)

        # Get keypoints for the current pair of images
        keypoints1 = keypoints_list[i]
        keypoints2 = keypoints_list[i + 1]

        # Draw matches between keypoints on the images
        matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, match, None)
        
        # Display the matched image
        plt.figure(figsize=(15, 7))
        plt.title(f"Matched Images {i + 1} to {i + 2}")
        plt.imshow(matched_image)
        plt.show()

if __name__ == "__main__":
    # Specify the root directory containing images
    root_dir = "C:/Vishal/Kaggele-Image-Matching-Challenge-2024/data/train"

    # Define the scene categories
    scene_categories = [
        "church",
        "dioscuri",
        "lizard",
        "multi-temporal-temple-baalshamin",
        "pond",
        "transp_obj_glass_cup",
        "transp_obj_glass_cylinder"
    ]

    # Iterate over each scene category
    for scene_category in scene_categories:
        print(f"Processing images for scene category: {scene_category}")

        # Start timing
        start_time = time.time()

        # Find image paths for the current scene category
        image_paths = find_image_paths(root_dir, scene_category)

        if not image_paths:
            print("No images found for this scene category.")
        else:
            # Detect and match features using OpenCV
            keypoints_list, matches = detect_and_match_features(image_paths)

            # Visualize matches
            visualize_matches(image_paths, keypoints_list, matches)

        # End timing
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"Time taken for scene category '{scene_category}': {elapsed_time:.2f} seconds")
        print("------------------------------------------------------------------")


Processing images for scene category: church


KeyboardInterrupt: 

In [1]:
def parse_camera_data(camera_file):
    camera_data = {}
    with open(camera_file, 'r') as file:
        lines = file.readlines()
        for line in lines[3:]:  # Skip the first three comment lines
            parts = line.strip().split(' ')
            camera_id = int(parts[0])
            model = parts[1]
            width = int(parts[2])
            height = int(parts[3])
            camera_data[camera_id] = {'model': model, 'width': width, 'height': height}
    return camera_data

def parse_image_data(image_file):
    image_data = {}
    with open(image_file, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 2):  # Process every two lines
            image_info = lines[i].strip().split(' ')
            image_id = int(image_info[0])
            camera_id = int(image_info[-2])
            name = image_info[-1]
            image_data[image_id] = {'camera_id': camera_id, 'name': name}
    return image_data

def parse_point3d_data(points3d_file):
    point3d_data = {}
    with open(points3d_file, 'r') as file:
        lines = file.readlines()
        for line in lines[3:]:  # Skip the first three comment lines
            parts = line.strip().split(' ')
            point3d_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            z = float(parts[3])
            point3d_data[point3d_id] = {'x': x, 'y': y, 'z': z}
    return point3d_data